In [22]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter 
from collections import *
import math

# Retrieve Data From Google Sheets

In [23]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [24]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# Reference: https://developers.google.com/sheets/api/quickstart/python
def read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE):
    creds = None
    # autogenerated
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    
    header = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                               range=HEADER_RANGE).execute()
    
    header_values = header.get('values', [])
    values = result.get('values', [])
    
    return values, header_values

In [25]:
SPREADSHEET_ID = '1AjynK9mMQTw58B_B8b_ZIip3fyUm-aoV7Pp21HziBb0'
RANGE_NAME = 'canto_codings!A2:AT'
HEADER_RANGE = 'canto_codings!A1:AT1'

data, header = read_google_sheets(SPREADSHEET_ID, RANGE_NAME, HEADER_RANGE)

df = pd.DataFrame(data, columns = header[0])

In [26]:
df.head()

,canto_coding_id,Culture,C_cid,cv_1,cv_2,cv_3,cv_4,cv_5,cv_6,cv_7,...,cv_34,cv_35,cv_36,cv_37,orv_1,orv_2,ensemble_value_id,ensemble_value_label,instrument_value_id,instrument_value_label
0,1,South Jakarta,17557,4,512,64,16,2,2,9216,...,16,16,128,16,1,6,1,All singers play instruments,6,5 kinds of instruments
1,2,Balinese,11507,4,2,2,16,2,2,2,...,2,2,128,16,1,0,null,,null,None
2,3,Balinese,11507,512,8,64,128,16,128,8192,...,16,16,2,128,7,3,1,All singers play instruments,3,2 kinds of instruments
3,4,Balinese,11507,4096,4096,4096,128,128,8192,8192,...,16,2,1024,16,7,3,1,All singers play instruments,1,No instruments
4,5,Sama,62459,4,256,4,16,2,2,16,...,16,16,128,16,1,1,null,,1,No instruments


# Mapping From Binary to Likert Scales

In [27]:
# codes represents the likert scale max values for each line
LIKERT = ["13","9","12","5","5","5","5","5","5","5","8","8_3","7", #13
          "8_6","4","13","7_2","11","7_3","5","6","6_2","5","7_4","5_2","6_3", #26
          "6_4","6_5","3","7_a","4_1","5","8_4","7_5","5","6_6","5"] #37

CODE_MAP = {
  "13": {
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "10": 10,
    "11": 11,
    "12": 12,
    "13": 13
  },
  "11": {
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "5": 4,
    "6": 5,
    "7": 6,
    "8": 7,
    "9": 8,
    "10": 9,
    "11": 10,
    "13": 11
  },
  "12": {
    "0": 0,
    "1": 1,
    "2": 2,
    "4": 3,
    "5": 4,
    "6": 5,
    "7": 6,
    "8": 7,
    "9": 8,
    "10": 9,
    "11": 10,
    "12": 11,
    "13": 12
  },
  "9": {
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "5": 4,
    "6": 5,
    "8": 6,
    "9": 7,
    "12": 8,
    "13": 9,
  },
  "8": {
    "0": 0,
    "1": 1,
    "3": 2,
    "5": 3,
    "6": 4,
    "9": 5,
    "10": 6,
    "11": 7,
    "13": 8
  },

  "8_2": {
    "0": 0,
    "1": 1,
    "3": 2,
    "5": 3,
    "6": 4,
    "7": 5,
    "9": 6,
    "11": 7,
    "13": 8
  },
  "8_3": {
    "0": 0,
    "1": 1,
    "3": 2,
    "4": 3,
    "5": 4,
    "7": 5,
    "9": 6,
    "11": 7,
    "13": 8
  },
  "8_4": {
    "0": 0,
    "1": 1,
    "3": 2,
    "4": 3,
    "6": 4,
    "7": 5,
    "8": 6,
    "10": 7,
    "13": 8
  },
  "8_5": {
    "0": 0,
    "1": 1,
    "3": 2,
    "4": 3,
    "6": 4,
    "7": 5,
    "8": 6,
    "10": 7,
    "13": 8
  },
  "8_6": {
    "0": 0,
    "1": 1,
    "3": 2,
    "5": 3,
    "6": 4,
    "7": 5,
    "9": 6,
    "11": 7,
    "13": 8
  },
  "7": {
    "0": 0,
    "1": 1,
    "3": 2,
    "5": 3,
    "6": 4,
    "9": 5,
    "11": 6,
    "13": 7
  },
  "7_2": {
    "0": 0,
    "1": 1,
    "4": 2,
    "7": 3,
    "9": 4,
    "10": 5,
    "13": 6,
    "14": 7
  },
  "7_3": {
    "0": 0,
    "1": 1,
    "3": 2,
    "4": 3,
    "7": 4,
    "9": 5,
    "11": 6,
    "13": 7
  },
  "7_4": {
    "0": 0,
    "1": 1,
    "3": 2,
    "5": 3,
    "7": 4,
    "9": 5,
    "11": 6,
    "13": 7
  },
  "7_5": {
    "0": 0,
    "1": 1,
    "3": 2,
    "4": 3,
    "7": 4,
    "8": 5,
    "10": 6,
    "13": 7
  },
  "6": {
    "0": 0,
    "1": 1,
    "4": 2,
    "7": 3,
    "10": 4,
    "11": 5,
    "13": 6,
  },
  "6_2": {
    "0": 0,
    "1": 1,
    "3": 2,
    "6": 3,
    "8": 4,
    "10": 5,
    "13": 6
  },
  "6_3": {
    "0": 0,
    "1": 1,
    "5": 2,
    "9": 3,
    "10": 4,
    "11": 5,
    "13": 6
  },
  "6_4": {
    "0": 0,
    "1": 1,
    "4": 2,
    "5": 3,
    "7": 4,
    "9": 5,
    "13": 6
  },
  "6_5": {
    "0": 0,
    "1": 1,
    "4": 2,
    "5": 3,
    "8": 4,
    "9": 5,
    "13": 6
  },
  "6_6": {
    "0": 0,
    "1": 1,
    "4": 2,
    "6": 3,
    "7": 4,
    "10": 5,
    "13": 6
  },
  "7_a": {
    "0": 0,
    "1": 1,
    "4": 2,
    "7": 3,
    "10": 4,
    "13": 5
  },
  "5": {
    "0": 0,
    "1": 1,
    "4": 2,
    "7": 3,
    "10": 4,
    "13": 5
  },
  "5_2": {
    "0": 0,
    "1": 1,
    "2": 1.5,
    "4": 2,
    "7": 3,
    "10": 4,
    "13": 5
  },
  "4": {
    "0": 0,
    "1": 1,
    "5": 2,
    "9": 3,
    "13": 4,
  },
  "4_1": {
    "0": 0,
    "1": 1,
    "7": 2,
    "10": 3,
    "13": 4,
  },
  "3": {
    "0": 0,
    "1": 1,
    "7": 2,
    "13": 3,
  }
}

# Utility Function

In [28]:
def camel_case_split(str): 
    words = [[str[0]]] 
    txt = ''
    
    for c in str[1:]: 
        if words[-1][-1].islower() and c.isupper(): 
            words.append(list(c)) 
        else: 
            words[-1].append(c) 
    word_list = [''.join(word) for word in words] 
    for i in range(len(word_list)):
        if i==0:
            txt += word_list[i]+' '
        else:
            txt += word_list[i].lower()+' '
    return txt.rstrip()

# Remove Samples of Instrumental Music without Vocals

In [29]:
df = df[df.cv_1 != "2"]
df = df[df.cv_1 != "0"]

# Read Metadata

In [30]:
## Read from Google Sheets
RANGE = 'metadata!A2:D'
HEADER_RANGE = 'metadata!A1:D1'
data, header = read_google_sheets(SPREADSHEET_ID, RANGE, HEADER_RANGE)

metadata = pd.DataFrame(data, columns = header[0])

In [31]:
def get_metadata(l, oc):
    filtered_data = metadata[metadata['line_num']==str(l)]
    original_codes = list(filtered_data['original_code'])
    descriptions = list(filtered_data['code_description'])
    shortnames = list(filtered_data['shortname'])
    for i in range(len(filtered_data)):
        if int(original_codes[i])==oc:
            return shortnames[i], descriptions[i]
    return 'Shortname Not Found', 'Description Not Found'

# Encode Data

In [32]:
def encode_data(line_num, line_max):
    row = []
    description = ''
    shortname = ''
    column_name = "cv_"+str(line_num)
    
    tmp_short = ''
    tmp_dsc = ''
 
    column = df[column_name]
    column_counter = dict(Counter(column))
    
    keys = list(column_counter.keys())
    values = list(column_counter.values())
    
    for i in range(len(keys)):
        original_1 = 0
        original_2 = None
        original_3 = None
        code_1 = 0
        code_2 = None
        code_3 = None
        display_code = 0
        var_id = ''
        if (keys[i] is None):
            continue
        elif(int(keys[i])==0):
            shortname = 'No Reading'
            description = 'No Reading'
        else:
            log = math.log2(int(keys[i]))
            if(log.is_integer()==False):
                closest = math.floor(log)
                diff = int(keys[i]) - (2**closest)
                if(math.log2(diff).is_integer()== False):
                    diff_2 = diff-(2**(math.floor(math.log2(diff))))
                    original_1 = int(closest)
                    original_2 = int(math.floor(math.log2(diff)))
                    original_3 = int(math.log2(diff_2))
                    display_code = round((original_1+original_2+original_3)/3,2)
                    var_id = str(line_num)+'_'+str(original_1)+'_'+str(original_2)+'_'+str(original_3)
                    
                    shortname = ''
                    description = ''
                    
                    tmp_short, tmp_dsc = get_metadata(line_num, original_1)
                    shortname += tmp_short+'And'
                    tmp_short, tmp_dsc = get_metadata(line_num, original_2)
                    shortname += tmp_short+'And'
                    tmp_short, tmp_dsc = get_metadata(line_num, original_3)
                    shortname += tmp_short
                    description = camel_case_split(shortname)
                    
                else:
                    original_1 = int(closest)
                    original_2 = int(math.log2(diff))
                    display_code = (original_1 + original_2)/2
                    var_id = str(line_num)+'_'+str(original_1)+'_'+str(original_2)
                    
                    shortname = ''
                    description = ''
                    
                    tmp_short, tmp_dsc = get_metadata(line_num, original_1)
                    shortname += tmp_short +'And'
                    tmp_short, tmp_dsc = get_metadata(line_num, original_2)
                    shortname += tmp_short
                    description = camel_case_split(shortname)
            else:
                original_1 = int(log)
                display_code = log
                var_id = str(line_num)+'_'+str(original_1)
                shortname, description = get_metadata(line_num, original_1)
        try:
            code_1 = CODE_MAP[LIKERT[line_num-1]][str(int(original_1))]
        except KeyError as e:
            print("CODE_1", " LIKERT:", LIKERT[line_num-1] ," LINE:", line_num, " KEY:", e)
        
        if(original_2):
            try:
                code_2 = CODE_MAP[LIKERT[line_num-1]][str(int(original_2))]
            except KeyError as e:
                print("CODE_2", " LIKERT:", LIKERT[line_num-1] ," LINE:", line_num, " KEY:", e)
        if(original_3):
            try:
                code_3 = CODE_MAP[LIKERT[line_num-1]][str(int(original_3))]
            except KeyError as e:
                print("CODE_3", " LIKERT:", LIKERT[line_num-1] ," LINE:", line_num, " KEY:", e)
        
        row.append(
            {
                "code":keys[i],
                "count": values[i], 
                "var_id_code": str(line_num)+"_"+keys[i], 
                "original_1": original_1, 
                "original_2":original_2, 
                "original_3": original_3, 
                "code_1": code_1, 
                "code_2": code_2, 
                "code_3": code_3, 
                "display_code": round(display_code/13,2),
                "var_id": var_id,
                "code_description": description,
                "shortname": shortname
            })
        
    row.sort(key=lambda x: x["display_code"])
    return(row)

# Execute

In [33]:
lines = {} #modify this for structure
x = 0
for i in range(37):
    line_key = "line_"+str(i+1)
    lines[line_key] = encode_data(i+1, LIKERT[i])
    
# line 12, 14, 17, 19, 22, 24, 25, 26, 27, 28, 31, 33, 34, 36

# Export to JSON

In [34]:
import json

with open('./output/output.json', 'w') as f:
    json.dump(lines, f)

# ERRORS

In [41]:
errors = []
error_df = pd.DataFrame(columns = df.columns)
# look at the output of the previous script and look for entries with 'Description Not Found' or 'ShortName Not Found'.
error_codes = ["36_64"]
for error in error_codes:
    obj = {}
    obj['line'] = error.split("_")[0]
    obj['code'] = error.split("_")[1]
    errors.append(obj)

In [42]:
error_report = []
for error in errors:
    line = list(df['cv_'+error['line']])
    songs = ""
    null_values = ""
    error_line = {}
    for i in range(len(line)):
        if line[i] == error['code']:
            try:
                songs+=", "+df[df["cv_"+error["line"]]==error["code"]]["canto_coding_id"].all()
            except KeyError as e:
                null_values+=str(e)+","
    error_line["line"] = error["line"]
    error_line["error"] = error["code"]
    error_line["canto_coding_ids"] = songs
    error_report.append(error_line)

In [43]:
with open('./output/error_report.json', 'w') as f:
    json.dump(error_report, f)